In [17]:
import sys
sys.path.append('../')
from lib.computational_graph_approach.layers import *
from lib.computational_graph_approach.utils import *
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import numpy as np
import cupy as cp
from IPython import embed

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Download data
MNIST_MEAN = 0.1307
MNIST_STDEV = 0.3081

def get_mnist(get_train_set):
  return torchvision.datasets.MNIST(
      root="data",
      train=get_train_set,
      download=True,
      # For demonstration
      transform=torchvision.transforms.Compose([
          torchvision.transforms.ToTensor(),
          torchvision.transforms.Normalize((MNIST_MEAN,), (MNIST_STDEV,))
      ])
  )

trn_set = get_mnist(get_train_set=True)
val_set = get_mnist(get_train_set=False)  # Technically the test set

In [15]:
class NN(Network):

    def __init__(self):
        self.layers = [
            FC_faster(28*28, 5, "relu"),
            FC_faster(5, 10)
        ]

    def __call__(self, inputs):
        output = []
        # inputs = cp.asarray(inputs)
        inputs = inputs.reshape(-1, 28*28)
        for input in inputs:
            x = self.layers[0](input)
            x = self.layers[1](x)
            output.append(x)
        return output

In [4]:
# Data formatting
NUM_DATA = 1000
trn_set_np = []
trn_set_np_labels = []
for d in range(NUM_DATA):
    trn_set_np.append(np.array(trn_set[d][0][0]))
    trn_set_np_labels.append(np.array(trn_set[d][1]))
trn_set_np = np.array(trn_set_np)
trn_set_np_labels = np.array(trn_set_np_labels)

In [54]:
def cross_entropy_loss(outputs, labels):
    loss = 0
    for out, l in zip(outputs, labels):
        numerator = np.sum([np.e ** o for o in out])
        denominator = np.e ** out[l]
        loss += (numerator/denominator).log()
    return loss


In [16]:
# Training
nn = NN()
timer = Timer()
BATCH_SIZE = 40

forwards = []
backwards = []

batch_data = np.split(trn_set_np, int(len(trn_set_np) / BATCH_SIZE))
batch_labels = np.split(trn_set_np_labels, int(len(trn_set_np) / BATCH_SIZE))
timer.start()
for d, l in zip(batch_data, batch_labels):
    # nn.zero_grad()

    # Forward pass
    timer.start()
    outputs = nn(d)
    # loss = cross_entropy_loss(outputs, l)
    forwards.append(timer.stop())

    # Backward pass
    # timer.start()
    # loss.backward()
    # backwards.append(timer.stop())

print(forwards)
print("Forward pass: {:.2f} ms".format(np.mean(np.array(forwards)) * 1000))
print(backwards)
print("Backward pass: {:.2f} ms".format(np.mean(np.array(backwards)) * 1000))


[0.6040589809417725, 0.574599027633667, 0.532639741897583, 0.7438745498657227, 0.5712816715240479, 0.594357967376709, 0.5847818851470947, 0.5714931488037109, 0.5440363883972168, 0.7349283695220947, 0.5636448860168457, 0.5640311241149902, 0.7457664012908936, 0.5789430141448975, 0.5840668678283691, 0.5764491558074951, 0.5505998134613037, 0.7159466743469238, 0.5588610172271729, 0.7578215599060059, 0.5821490287780762, 0.5800528526306152, 0.5513007640838623, 0.7262320518493652, 0.5630800724029541]
Forward pass: 610.20 ms
[]
Backward pass: nan ms


/home/sujay/miniconda3/envs/567-final-project/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/sujay/miniconda3/envs/567-final-project/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
